# The fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

true

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

["a", "b", "c", "d", "e", "f"]
["1", "a", "c", "e"]


In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]


["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# Wcode Class

In [79]:
class Wcode < Hash
  def initialize(word = Group::Identity)
    @words = [word]
    #---
    Rseq.each{|x| self.store(x.char, [])}
    if word != Group::Identity
      word.flatten.factors.each_with_index do |c, i|
        k = i
        self[c.char].insert(-1, 2*k)
        self[c.inverse.char].insert(0,2*k+1)
      end
    end
  end
  attr_reader :words
  #---
  def show()
    self.values.map(&:to_s).join
  end
  def to_a()
    self.values.flatten
  end
  def size()
    self.to_a.size
  end
  def copy()
    wc = self.class.new
    wc.each_key{|key| wc[key] = self[key].dup}
    wc
  end
  #---
  def narrow()
    wc = self.copy
    sorted = wc.values.flatten.sort
    wc.each_key do |key|
      wc[key].map!{|v| sorted.index(v)}
    end
    wc
  end
  #---
  def to_word()
    chars = (0..self.size/2).to_a.map{|i| self.map{|key, arr| key if arr.include?(2*i)}.join}
    Word.new(chars.join)
  end
  #---
  def product(another)
    wc = self.copy
    size = self.size
    wc.map do |k, v|
      another[k].each do |n|
        n = n + self.size
        (n.odd?) ? v = [n]+v : v << n 
      end
      wc[k] = v
    end
    wc
  end
  #---
  def devide(odd_nums)
    facs = []
    facs[0] = self.map{|key, arr| [key, arr.select{|x| x <= odd_nums[0] or odd_nums[1] < x}] }.to_h
    facs[1] = self.map{|key, arr| [key, arr.select{|x| odd_nums[0] < x and x <= odd_nums[1]}] }.to_h
    facs.map! do |f|
      wc = self.class.new
      wc.each_key{|key| wc[key] = f[key]}
    end
    return facs
  end
  #---
  ## Turaev cobracket for Wcodes
  def cobracket()
    odd_pairs=(1..self.size/2).to_a.map{|x| 2*x-1}.combination(2)
    factors = odd_pairs.map do |odds|
      divs = odds.map{|odd| Division.new(self, odd)}
      linking = divs[0].linking(divs[1])
      factors = self.devide(odds).map(&:narrow)
      #---
      {odds: odds, linking: linking, factors: factors }
    end
    return factors.select{|prd| prd[:linking] != 0}
  end  
end; nil

#-----------------------------------
class Division
  def initialize(wc, odd)
    @wc = wc
    @odd = odd
    wcvf = wc.values.flatten
    @indices = [wcvf.index(@odd), wcvf.index((@odd+1)%(@wc).size)]
    ids = @indices.sort
    @interior = wcvf[ids[0]+1..ids[1]-1]
    @sign = (@indices == @indices.sort) ? 1 : -1
  end
  attr_reader :indices, :interior, :sign
  
  def linking(another)
=begin
    if (self.index(odd1) - self.index(odd1+1) + 1).even?
      # [o,e,*,*] / [o,*,*,e] / [*,o,e,*] / [*,*,o,e] (and reverse 'o' and 'e')
      return 0
    else
      # [o1,e2,e1,o2] --> linking = +1 and [o2,o1,e2,e1] --> linking = +1
      # The sign of linking is change by (o1<->e1) and (e2<->o2)
      idxs = [self.index(odd1), self.index(odd1+1)] 
      sign0 = (idxs[0] < idxs[1]) ? 1 : -1
      rng = idxs.sort
      sign2 = self[rng[0]..rng[1]][1].odd? ? 1 : -1
      #---
      sign = sign0*sign2
    end
=end  
  end
end;nil

In [78]:
w = Word.new(a[1], a[2], b[1].inverse)
wc = Wcode.new(w)
p w.show, wc.show
printf("===\n")

wcf = wc.values.flatten

(1..wc.size/2).map{|n| 2*n-1}.each do |odd|
  div = Division.new(wc, odd)
  ids = div.indices.sort
  p odd #, div.indices, 
  p div.interior, div.sign
end

nil

"acB"
"[0][4][1][5][2][][3][][][][][]"
===
1
[5]
1
3
[1, 5, 2]
-1
5
[4, 1]
-1


In [ ]:
wc = wc_arr.inject(:product)
p wc.to_word.show, wc.show
printf("===\n")

wc.cobracket.each do |h|
  p "order : #{h[:order]}"
  p "linking : #{h[:linking]}"
  p "factors : " + h[:factors].map(&:show).join(' , ')
  prd = h[:factors].inject(&:product)
  p prd.show
  p h[:factors].map(&:to_word).map(&:show).join(" \u{2227} ")
  p h[:factors].map(&:to_word).map(&:cyclic_reduce).map(&:show).join(" \u{2227} ")
#  p prd.to_word.show
  printf("-----\n")
end
nil

## Turaev cobraket

In [ ]:
def cobracket(simple_words_arr)
  wa = simple_words_arr
  code = Wcode.new(wa)
  #---
  cob = code.cobracket
  
  unless cob.empty?
    cob.each do |h|
      h[:words] = h[:factors].map do |f|
        evens = f.values.flatten.select{|v| v.even?}.sort
        evens.map{|e| wa[e/2]}
      end
    end
  end

  return cob
end;nil

# Execution

In [ ]:
a1, a2 = Word.new(a[1]), Word.new(a[2])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten
samples = [
  [a1],
  [a1, Word.new(b[2])],
  [a1, Word.new(b[2]).inverse],
  #---
  (1..4).map{|i| a1},
  #---
  [a1, (comms[2].inverse)],
  [a1, comms[2]],
  [a1] + (1..3).map{|i| comms[2]},
  [a1] + (1..3).map{|i| a2},
  [a1] + (1..3).map{|i| Word.new(a[2], b[2].inverse)},
  [a1, comms[2], c, comms[2], c.inverse],
  #---
  [a1, Group.commutator(a[2], a[3])],
  [a1, comms[2]*comms[3], Group.commutator(a[2], a[3]), (comms[2]*comms[3]).inverse]
  ]

samples.each do |wa|
  ws = wa.inject(:*).show.gsub(/[()]+/, '.').gsub(/^[.]|[.]$/, '')
  terms = cobracket(wa).map do |h|
    coeff = (h[:linking]==1) ? '' : '(-1)'
    coeff + h[:words].map{|a| a.inject(:*).cyclic_reduce.show }.join(" \u{2227} ")
  end
  p "#{ws}  |--\u{03B4}--> #{terms.join(' + ')}"
end;nil

"a  |--δ-->  0"
"a.d  |--δ-->  a ∧ d"
"a.D  |--δ-->  0"
"a.a.a.a  |--δ-->  (-1) aa ∧ aa"
"abA  |--δ-->  (-1) 1 ∧ b"
"abAB  |--δ-->  0"
"a.dcDC  |--δ-->  a ∧ dcDC"
"a.cdCD  |--δ-->  0"
"a.cdCD.cdCD.cdCD  |--δ-->  (-1) acdCDcdCD ∧ cdCD + (-1) acdCD ∧ cdCDcdCD"
"a.c.c.c  |--δ-->  (-1) acc ∧ c + (-1) ac ∧ cc"
"a.cD.cD.cD  |--δ-->  (-1) acDcD ∧ cD + (-1) acD ∧ cDcD"
"a.cdCD.abA.cdCD.aBA  |--δ-->  (-1) acdCD ∧ cdCD + (-1) cdCD ∧ cdCDa"
"a.ceCE  |--δ-->  aceC ∧ E + (-1) ace ∧ CE + ac ∧ C"
"a.cdCD.efEF.ceCE.feFEdcDC  |--δ-->  acdCDefEFceCfeFEdcDC ∧ E + (-1) acdCDefEFcefeFEdcDC ∧ CE + acdCDefEFcfeFEdcDC ∧ C"

In [ ]:
warr = samples[-3]
wc = Wcode.new(warr)
p wc.show, wc.words.map(&:show)
nil

In [ ]:
cob = cobracket(warr)
cob.each do |h|
  p "order: #{h[:order]}"
  p "linking: #{h[:linking]}"
  p "factors: #{h[:factors].map(&:show)}"
  p "words: #{h[:words].map{|a| a.inject(:*).show}}"
  p h[:words].map{|a| a.inject(:*).cyclic_reduce.show }.join(" \u{2227} ")
  printf("-- -- --\n")
end
nil

### TODO
$[a_{1},b_{1}]$ の処理において、
* [x] [5, 0, 6, 7] の sign が正しくないようだ。
* [x] a(x)A [ $a_{1}\otimes a_{1}^{-1}$ ] の集計もおかしい。
* [x] $a_{1}^{3}$ の計算がエラーになる。
* [ ] simple subword たちによる計算を実装する。